In [6]:
from SMT_model import *
from time import time as time_clock

In [7]:
#Demo with instance 01
Path = ('../Instances/inst04.dat')

timeout = 300


In [8]:
#Read the file from the path
with open(Path, 'r') as file:
    data = file.read()

instance = data_parsing(data)
print(instance)

{'m': 8, 'n': 10, 'l': [200, 180, 160, 185, 180, 200, 160, 200], 's': [10, 25, 18, 16, 14, 16, 24, 19, 23, 19], 'D': [[0, 56, 86, 77, 81, 128, 107, 154, 70, 93, 53], [56, 0, 79, 31, 62, 87, 61, 107, 37, 37, 24], [86, 79, 0, 109, 17, 43, 110, 68, 43, 69, 55], [87, 31, 109, 0, 92, 116, 30, 77, 66, 40, 55], [81, 62, 17, 92, 0, 47, 93, 82, 26, 52, 38], [128, 87, 43, 116, 47, 0, 117, 52, 58, 76, 75], [116, 61, 110, 30, 93, 117, 0, 65, 67, 41, 63], [163, 107, 78, 77, 82, 52, 65, 0, 93, 70, 110], [70, 41, 43, 66, 26, 73, 67, 93, 0, 26, 17], [93, 37, 69, 40, 52, 76, 41, 70, 26, 0, 40], [53, 24, 55, 54, 38, 75, 63, 110, 17, 40, 0]]}


In [16]:
def run_smt(_, instance, timeout, sb=False):
    generation_start_time = time_clock()

    o, x, max_distance = stm_model(instance, timeout, sb)
    generation_duration = time_clock() - generation_start_time
    o.set("timeout", int(timeout - generation_duration) * 1000)

    obj = o.minimize(max_distance)
    res = o.check()
    if res in [sat, unknown]:
        result_formatted = format_solution(instance, o.model(), x)
        return result_formatted, res==sat, obj.value()
    elif res == unsat:
        return "unsat"
    else:
        return "unknown"

In [17]:
print(run_smt(_, instance, timeout, False))

[x_0_4 = 10,
 x_2_3 = 10,
 x_7_1 = 10,
 x_3_3 = 10,
 s = Store(Store(Store(Store(Store(Store(Store(Store(Store(K(Int,
                                        19),
                                        1,
                                        25),
                                        10,
                                        0),
                                        5,
                                        16),
                                        8,
                                        23),
                                   3,
                                   16),
                             0,
                             10),
                       4,
                       14),
                 2,
                 18),
           6,
           24),
 x_2_10 = 10,
 x_4_10 = 10,
 x_5_4 = 10,
 x_4_9 = 10,
 x_1_10 = 10,
 x_5_8 = 10,
 x_2_5 = 9,
 x_1_4 = 10,
 x_5_10 = 10,
 x_1_6 = 10,
 x_3_10 = 10,
 x_2_9 = 10,
 x_6_7 = 10,
 x_7_7 = 10,
 x_6_3 = 10,
 x_4_8 = 4,
 x_3

x_0_5 = 6,
 x_1_3 = 5,
 max_distance = 14,
 x_1_0 = 6,
 y_1 = 14,
 load_1 = 10,
 y_0 = 13,
 x_0_0 = 6,
 load_0 = 14,
 x_0_6 = 6,
 x_1_6 = 6]
([[1, 3, 4], [2, 5, 6]], True)